In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
####################################################################
#        Please comment on my code as I am a novice to ML          #
####################################################################
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#reading input file
df_covid = pd.read_csv("../input/covid19-bharatbiotech-tweets/BharatBioTech-Refined - Covaxin.csv")

In [ ]:
df_covid.head()

In [ ]:
df_covid.describe()

In [ ]:
# Checking for null values
sns.heatmap(df_covid.isnull(), yticklabels = False, cbar = False, cmap="Blues")

In [ ]:
# Visualisations
plt.figure(figsize=[10,5])
plt.subplot(131)
sns.countplot(x='username', data=df_covid)
plt.subplot(132)
sns.countplot(x='name', data=df_covid)
plt.subplot(133)
sns.countplot(x='language', data=df_covid)

In [ ]:
# Dropping the columns username ane name based on the visualisation
df_covid.drop(['username','name','language'],axis=1,inplace=True)

In [ ]:
df_covid.head()

In [ ]:
# Adding a column to record the tweet_type based on replies, retweets and likes
df_covid['tweet_type'] = pd.Series([0 for x in range(len(df_covid.index))])

In [ ]:
df_covid.head()

In [ ]:
# function where categorise if the tweet is popular or not 1 - trending tweet and 0 - Not trending tweet
# Note: Threshold can be changed and impact can be seen in the model later (prediction if the tweet will trend or not)
def update_tweet_status(data):
    replies = data[0]
    retweets = data[1]
    likes = data[2]
    if likes > 25:
        return 1
    else: 
        if retweets > 5:
            return 1
        else: 
            if replies > 10:
                return 1
            else:
                return 0

In [ ]:
df_covid['tweet_type'] = df_covid[['replies_count','retweets_count','likes_count']].apply(update_tweet_status,axis=1)

In [ ]:
plt.figure(figsize=[5,5])
sns.countplot(x='tweet_type', data=df_covid)

In [ ]:
#Dropping the 3 columns since we now have tweet_type
df_covid.drop(['replies_count','retweets_count','likes_count','hashtags'],axis=1,inplace=True)

In [ ]:
df_covid.head()

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
corpus = []
for i in range(0,238):
  twt = re.sub('[^a-zA-Z]', ' ', df_covid['tweet'][i])
  twt = twt.lower()
  twt = twt.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  twt = [ps.stem(word) for word in twt if not word in set(all_stopwords)]
  twt = ' '.join(twt)
  corpus.append(twt)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500)
X = vectorizer.fit_transform(corpus).toarray()
y = df_covid['tweet_type'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
y_predict = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
cm = confusion_matrix(y_test,y_predict)
sns.heatmap(cm,annot=True)

In [ ]:
print(classification_report(y_test, y_predict))

In [ ]:
# We get a decent accuracy which means any new tweet we get we can classify if it will trend or not.